#### **MiniProject 7 - Open Crime UK**

####**Part II**
The two .csv files discussed for this part are located in this google drive folder.
The file "Employee_start_time_end_time.csv" contains the start and end times for employees' daily work shifts. The file "hour_range_slot_for_pay.csv" specifies the pay rate applicable based on the number of hours worked for a particular day. For example, if an employee works for 4 hours on a given day, the applicable pay rate is $10 per hour, as 4 hours falls within the range of 1 to 5 hours (see row 1 in the "hour_range_slot_for_pay.csv" file). If an employee works for 7 hours on a given day, the pay rate is $12 per hour, as it falls within the range of 6 to 10 hours (see row 2 in the "hour_range_slot_for_pay.csv" file). Using these criteria, you need to calculate the total pay in USD for each employee based on the provided information from the two CSV files.
Credit Card Fraud Detection: Credit Card Fraud classification using Random Forest Classifier. Column value to be classified is in column "Is_fraud" and all other columns are the features. Use two-fold cross validation and evaluation metrics: Precision, Recall, F1-score, Accuracy [wiki]. Also print confusion matrix [wiki]. Refer basic notebook template for some code and also internet.

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate

In [8]:
from google.colab import files
uploaded = files.upload()

Saving employee_start_time_end_time.csv to employee_start_time_end_time.csv
Saving hour_range_slot_for_pay.csv to hour_range_slot_for_pay.csv


In [9]:
employee_data = pd.read_csv('/content/employee_start_time_end_time.csv')
pay_rate_data = pd.read_csv('/content/hour_range_slot_for_pay.csv')

In [10]:
employee_data.head(5)

,employee_id,start_time,end_time
0,1,1-Mar-2022 09:10,1-Mar-2022 05:10
1,2,1-Mar-2022 08:10,1-Mar-2022 05:10
2,3,1-Mar-2022 06:00,1-Mar-2022 05:10
3,4,1-Mar-2022 06:10,1-Mar-2022 05:10
4,5,1-Mar-2022 05:10,1-Mar-2022 05:10


In [ ]:
pay_rate_data

,daily_hour_min_range,daily_hour_max_range,pay_per_hour_usd
0,1,5,10
1,6,10,12
2,11,100,20


####1) The file "Employee_start_time_end_time.csv" contains the start and end times for employees' daily work shifts. The file "hour_range_slot_for_pay.csv" specifies the pay rate applicable based on the number of hours worked for a particular day. For example, if an employee works for 4 hours on a given day, the applicable pay rate is $10 per hour, as 4 hours falls within the range of 1 to 5 hours (see row 1 in the "hour_range_slot_for_pay.csv" file). If an employee works for 7 hours on a given day, the pay rate is $12 per hour, as it falls within the range of 6 to 10 hours (see row 2 in the "hour_range_slot_for_pay.csv" file). Using these criteria, you need to calculate the total pay in USD for each employee based on the provided information from the two CSV files.


In [31]:
import pandas as pd
from datetime import datetime, timedelta

employee_data.rename(columns={'employee_id': 'Employee_ID', 'start_time': 'Start_Time', 'end_time': 'End_Time'}, inplace=True)
pay_rate_data.rename(columns={'daily_hour_min_range': 'Min_Hours', 'daily_hour_max_range': 'Max_Hours', 'pay_per_hour_usd': 'Pay_Rate'}, inplace=True)

In [32]:
# Convert Start_Time and End_Time to datetime
employee_data['Start_Time'] = pd.to_datetime(employee_data['Start_Time'], errors='coerce')
employee_data['End_Time'] = pd.to_datetime(employee_data['End_Time'], errors='coerce')

# Drop rows with invalid datetime values
employee_data.dropna(subset=['Start_Time', 'End_Time'], inplace=True)

# Adjust End_Time if it's earlier than Start_Time (assuming overnight shifts)
employee_data['End_Time'] = employee_data.apply(lambda row: row['End_Time'] + timedelta(days=1) if row['End_Time'] < row['Start_Time'] else row['End_Time'], axis=1)

# Calculate Work_Hours correctly
employee_data['Work_Hours'] = (employee_data['End_Time'] - employee_data['Start_Time']).dt.total_seconds() / 3600  # Convert seconds to hours


In [13]:
def get_pay_rate(hours_worked):
    for _, row in pay_rate_data.iterrows():
        if row['Min_Hours'] <= hours_worked <= row['Max_Hours']:
            return row['Pay_Rate']
    return 0  # Default pay rate if no range matches

# Apply pay rate calculation
employee_data['Pay_Rate'] = employee_data['Work_Hours'].apply(get_pay_rate)

# Calculate Total Pay
employee_data['Total_Pay'] = employee_data['Work_Hours'] * employee_data['Pay_Rate']

# Save results to a new CSV file
employee_data[['Employee_ID', 'Start_Time', 'End_Time', 'Work_Hours', 'Pay_Rate', 'Total_Pay']].to_csv("Employee_Pay_Report.csv", index=False)

# Display results
print(employee_data[['Employee_ID', 'Start_Time', 'End_Time', 'Work_Hours', 'Pay_Rate', 'Total_Pay']])


    Employee_ID          Start_Time            End_Time  Work_Hours  Pay_Rate  \
0             1 2022-03-01 09:10:00 2022-03-02 05:10:00   20.000000        20   
1             2 2022-03-01 08:10:00 2022-03-02 05:10:00   21.000000        20   
2             3 2022-03-01 06:00:00 2022-03-02 05:10:00   23.166667        20   
3             4 2022-03-01 06:10:00 2022-03-02 05:10:00   23.000000        20   
4             5 2022-03-01 05:10:00 2022-03-01 05:10:00    0.000000         0   
5             6 2022-03-01 08:10:00 2022-03-02 05:10:00   21.000000        20   
6             7 2022-03-01 07:10:00 2022-03-02 05:10:00   22.000000        20   
7             1 2022-03-02 06:10:00 2022-03-03 05:10:00   23.000000        20   
8             2 2022-03-02 05:10:00 2022-03-02 05:10:00    0.000000         0   
9             3 2022-03-02 08:10:00 2022-03-03 05:10:00   21.000000        20   
10            4 2022-03-02 07:10:00 2022-03-03 05:10:00   22.000000        20   
11            5 2022-03-02 0

In [16]:
print(tabulate(employee_data[['Employee_ID', 'Work_Hours', 'Pay_Rate', 'Total_Pay']], headers='keys', tablefmt='psql'))


+----+---------------+--------------+------------+-------------+
|    |   Employee_ID |   Work_Hours |   Pay_Rate |   Total_Pay |
|----+---------------+--------------+------------+-------------|
|  0 |             1 |      20      |         20 |     400     |
|  1 |             2 |      21      |         20 |     420     |
|  2 |             3 |      23.1667 |         20 |     463.333 |
|  3 |             4 |      23      |         20 |     460     |
|  4 |             5 |       0      |          0 |       0     |
|  5 |             6 |      21      |         20 |     420     |
|  6 |             7 |      22      |         20 |     440     |
|  7 |             1 |      23      |         20 |     460     |
|  8 |             2 |       0      |          0 |       0     |
|  9 |             3 |      21      |         20 |     420     |
| 10 |             4 |      22      |         20 |     440     |
| 11 |             5 |      20      |         20 |     400     |
| 12 |             6 |   

###2) Credit Card Fraud Detection: Credit Card Fraud classification using Random Forest Classifier. Column value to be classified is in column "Is_fraud" and all other columns are the features. Use two-fold cross validation and evaluation metrics: Precision, Recall, F1-score, Accuracy [wiki]. Also print confusion matrix [wiki]. Refer basic notebook template for some code and also internet.
Data: [Kaggle]
Expected output:
empoyee_id,total_pay_usd

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, classification_report

In [45]:
file_path = "/content/fraud test.csv"
data = pd.read_csv(file_path)
data.head(5)

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,21/06/2020 12:14,2.291160e+15,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,...,33.9659,-80.9355,333497,Mechanical engineer,19/03/1968,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,1,21/06/2020 12:14,3.573030e+15,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,...,40.3207,-110.4360,302,"Sales professional, IT",17/01/1990,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,2,21/06/2020 12:14,3.598220e+15,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,...,40.6729,-73.5365,34496,"Librarian, public",21/10/1970,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0
3,3,21/06/2020 12:15,3.591920e+15,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,...,28.5697,-80.8191,54767,Set designer,25/07/1987,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0
4,4,21/06/2020 12:15,3.526830e+15,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,...,44.2529,-85.0170,1126,Furniture designer,06/07/1955,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0


In [24]:
target_column = "is_fraud"
if target_column not in data.columns:
    raise ValueError("Target column 'Is_fraud' not found in the dataset")

In [37]:
# Separate features and target variable
X = data.drop(columns=[target_column])
y = data[target_column]

X = X[y.notna()]  # Keep rows in X where y is not NaN
y = y[y.notna()]  # Keep rows in y where y is not NaN

# Convert timestamp columns to numerical format
for col in X.select_dtypes(include=['datetime64']).columns:
    X[col] = X[col].astype(np.int64) // 10**9  # Convert to Unix timestamp (seconds)

# Identify and handle non-numeric columns
for col in X.select_dtypes(include=['object']).columns:
    # Instead of checking for 'Baroda', apply one-hot encoding to all object columns
    print(f"Column '{col}' is non-numeric. Applying one-hot encoding.")
    X = pd.get_dummies(X, columns=[col], prefix=[col], drop_first=True)

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Initialize Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Perform two-fold cross-validation
y_pred_cv = cross_val_predict(rf_model, X, y, cv=2)

Column 'city' is non-numeric. Applying one-hot encoding.
Column 'state' is non-numeric. Applying one-hot encoding.


In [38]:
precision = precision_score(y, y_pred_cv, average='binary')
recall = recall_score(y, y_pred_cv, average='binary')
f1 = f1_score(y, y_pred_cv, average='binary')
accuracy = accuracy_score(y, y_pred_cv)
conf_matrix = confusion_matrix(y, y_pred_cv)

In [44]:
metrics_data = [
    ["Precision", precision],
    ["Recall", recall],
    ["F1-score", f1],
    ["Accuracy", accuracy]
]

# Display the metrics table
print(tabulate(metrics_data, headers=["Metric", "Value"], tablefmt="psql"))

# Display the confusion matrix
print("\nConfusion Matrix:")
print(tabulate(conf_matrix, tablefmt="psql"))  # Using tabulate for better formatting

# Display the classification report
print("\nClassification Report:")
print(classification_report(y, y_pred_cv))

+-----------+------------+
| Metric    |      Value |
|-----------+------------|
| Precision | 0.00309109 |
| Recall    | 0.266286   |
| F1-score  | 0.00611123 |
| Accuracy  | 0.634176   |
+-----------+------------+

Confusion Matrix:
+--------+-------+
| 131148 | 75145 |
|    642 |   233 |
+--------+-------+

Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      0.64      0.78    206293
         1.0       0.00      0.27      0.01       875

    accuracy                           0.63    207168
   macro avg       0.50      0.45      0.39    207168
weighted avg       0.99      0.63      0.77    207168

